In [27]:
import glob
import json
import pandas as pd

In [29]:
def generate_result(result_dict, result_files):

  for i in result_files:
    with open(i, "r") as f:
      data = json.load(f)
      for key, value in data.items():
        result_dict[str(key).zfill(4)]+=value[0]

  # some result cases have more than one feedback
  for key, value in result_dict.items():
        if len(value) >3:
          temp = value[:2]
          temp.append(value[2:])
          result_dict[key] = temp

  df_result = pd.DataFrame.from_dict(result_dict, orient='index',columns=['difficulty', 'result', 'feedback'])
  df_diff = df_result.groupby(['difficulty', 'result']).count()
  df_diff['percent'] = df_diff.groupby('difficulty', group_keys=False)['feedback'].apply(lambda x: (round(x*100/x.sum(), 2)))
  df_diff['percent'] = pd.Series(["{0:.2f}%".format(val)  for val in df_diff['percent']], index = df_diff.index)

  df_all = df_result.groupby(['result']).agg({'feedback': 'count'})
  df_all['percent'] = round(df_all['feedback']*100/df_all['feedback'].sum(), 2)
  df_all['percent'] = pd.Series(["{0:.2f}%".format(val)  for val in df_all['percent']], index = df_all.index)

  return df_diff, df_all

In [30]:
# Read difficulty json
with open("json_files/difficulty.json", "r") as f:
  result_dict = json.load(f)

# Get result path
RESULT_PATH = 'json_files/original'
result_files = glob.glob(RESULT_PATH+"/*_results.json")

In [31]:
df_diff, df_all = generate_result(result_dict, result_files)

In [32]:
df_all

,feedback,percent
result,,
-2,57,1.14%
-1,866,17.32%
False,2864,57.28%
True,1213,24.26%


In [33]:
df_diff

feedback percent
difficulty   result                  
competition  -2            19   1.90%
             -1           267  26.70%
             False        651  65.10%
             True          63   6.30%
interview    -2            27   0.90%
             -1           495  16.50%
             False       1838  61.27%
             True         640  21.33%
introductory -2            11   1.10%
             -1           104  10.40%
             False        375  37.50%
             True         510  51.00%